### **AI Agent**

> **Agent**는 **스스로 판단해서 행동을 결정하는 AI 시스템**

일반적인 LLM은 정해진 프롬프트에 따라 한 번만 응답하지만,  
**Agent는 여러 단계로 추론**하면서 **외부 도구를 호출하거나, 정보를 수집하고, 그 결과를 바탕으로 다시 행동을 결정** 함

#### **Agent의 구성요소**

| 구성요소 | 설명 |
|----------|------|
| **LLM (Think)** | 주어진 목표를 이해하고, 어떤 행동을 할지 판단 |
| **Tools (Act)** | 계산기, 검색엔진, 데이터베이스 질의 등 실행 가능한 도구 |
| **Memory (Optional)** | 이전 대화나 작업 결과를 기억 |
| **Planner/Executor** | (고급) 작업을 계획하고 순차적으로 실행 |


### **tools**

> **Tools는 Agent가 사용할 수 있는 “실제 기능”.**

- 계산기
- 웹 검색
- 데이터베이스 질의
- API 호출
- 사전 정의된 파이썬 함수

Agent가 직접 실행할 수 있도록 연결해주는 "인터페이스"

LLM은 **외부 세계의 정보**를 알 수 없음.  
근데 Agent가 진짜 유용하려면 **외부 도구를 통해 뭔가 “행동”할 수 있어야** 함.

LangChain은 LLM이 직접 사용할 수 있는 **툴 인터페이스**를 구현

LangChain에서는 모든 Tool이 내부적으로는 **`Runnable`**로 작동
체인처럼 연결하거나, 병렬 실행도 가능하고, tracing 가능

### **요약**

| 개념 | 설명 |
|------|------|
| `tool` | Agent가 사용할 수 있는 "기능 함수" |
| 목적 | LLM이 외부 행동(계산, 검색, API 등)을 할 수 있게 함 |
| 형태 | 일반 함수에 `@tool`을 붙이면 됨 |
| 예 | 계산기, 날씨 정보, DB 질의 등 |

- [LangChain Tools/Toolkits](https://python.langchain.com/docs/integrations/tools/)

### **빌트인 도구(built-in tools)**

랭체인에서 제공하는 사전에 정의된 도구(tool) 와 툴킷(toolkit) 을 사용할 수 있음

tool 은 단일 도구를 의미하며, toolkit 은 여러 도구를 묶어서 하나의 도구로 사용할 수 있음

### **Python REPL 도구**

Python 코드를 REPL(Read-Eval-Print Loop) 환경에서 실행하기 위한 클래스를 제공
- PythonREPLTool

**설명**

- Python 셸 환경을 제공
- 유효한 Python 명령어를 입력으로 받아 실행
- 결과를 보려면 print(...) 함수를 사용

**주요 특징**

- sanitize_input: 입력을 정제하는 옵션 (기본값: True)
- python_repl: PythonREPL 인스턴스 (기본값: 전역 범위에서 실행)

**사용 방법**

- PythonREPLTool 인스턴스 생성
- run 또는 arun, invoke 메서드를 사용하여 Python 코드 실행

**입력 정제**

- 입력 문자열에서 불필요한 공백, 백틱, 'python' 키워드 등을 제거

In [1]:
from langchain_experimental.tools import PythonREPLTool

# 파이썬 코드를 실행하는 Tools
python_tool = PythonREPLTool()

# Tool을 사용하여 파이썬 코드 실행
code = '''
for i in range(1, 9):
    for j in range(1, 9):
        print(f"{i} * {j} = {i * j}")
'''
        
print(python_tool.invoke(code))

Python REPL can execute arbitrary code. Use with caution.


300



### **LLM에게 코드 작성 요청 & Tools로 실행**

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda
from langchain_experimental.tools import PythonREPLTool

In [27]:
prompt = PromptTemplate.from_template(
    '''
You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. 
You follow the PEP8 style guide.
Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.
 
user:           
{input}
'''
)

def print_and_execute(code, debug=False):
    if debug:
        print("CODE:")
        print(code)
        
    return python_tool.invoke(code)

llm = ChatOpenAI(model="gpt-4o")

chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

print(chain.invoke('구구단 중 1단부터 2단까지 출력하는 파이썬 코드를 작성하시오.'))

[9, 23, 27, 38, 42, 43]



### **검색 API 도구**

Tavily 검색 API를 활용하여 검색 기능을 구현하는 도구(1,000 API credits / month)

`TavilySearchResults`와 `TavilyAnswer`  => Tools용 

`TavilySearchAPIRetriever` => RAG 검색기용

### TavilySearchResults

**설명**
- Tavily 검색 API를 쿼리하고 JSON 형식의 결과를 반환
- 포괄적이고 정확하며 신뢰할 수 있는 결과에 최적화된 검색 엔진
- 현재 이벤트에 대한 질문에 답변할 때 유용합니다.

**주요 매개변수**
- `max_results` (int): 반환할 최대 검색 결과 수 (기본값: 5)
- `search_depth` (str): 검색 깊이 ("basic" 또는 "advanced")
- `include_domains` (List[str]): 검색 결과에 포함할 도메인 목록
- `exclude_domains` (List[str]): 검색 결과에서 제외할 도메인 목록
- `include_answer` (bool): 원본 쿼리에 대한 짧은 답변 포함 여부
- `include_raw_content` (bool): 각 사이트의 정제된 HTML 콘텐츠 포함 여부
- `include_images` (bool): 쿼리 관련 이미지 목록 포함 여부

**반환 값**
- 검색 결과를 포함하는 JSON 형식의 문자열(url, content)


[Tavily 참고](https://velog.io/@woody_ahn/Tavily-LLM%EC%9D%84-%EC%9C%84%ED%95%9C-%EC%B5%9C%EC%A0%81%ED%99%94%EB%90%9C-%EA%B2%80%EC%83%89-API)

In [32]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Tools 생성
tool = TavilySearchResults(
    max_results=3,
    include_answer=True,
    include_raw_content=True,
    # include_images = True,
    # search_depth = 'basic' or 'advanced'
    include_domains = ['google.com', 'naver.com'],  # 포함할 도메인
    # exclude_domains = ['daum.net'],   # 제외할 도메인
)

In [33]:
# 도구 실행
tool.invoke({"query": "에스티이노베이션이라는 회사에 대해서 알려주세요"})

[{'title': '2차전지 관련주 한국에 어떤 기업이 있나요? - 네이버 지식iN',
  'url': 'https://m.kin.naver.com/qna/dirs/40102/docs/336522725?d1id=4&qb=7JSo7JWE7J207JeQ7Iqk&enc=utf8&section=kin.qna&rank=108&search_sort=2&spq=0',
  'content': '탈퇴한 사용자 답변\n2019.09.24\n메뉴 더보기\n접기\n더보기 2차전지 관련 회사들 리스트입니다\u200b한국아트라스비엑스,디아이티,엠플러스,켐트로스,삼진엘앤디,애경유화,테이팩스,넥스트아이,영화테크,일진머티리얼즈,필옵틱스,에코프로비엠,에코프로,유에스티,포스코케미칼,삼성SDI,상아프론테크,후성,솔브레인,에이에프더블류,리켐,신흥에스이씨,피엔티,천보,브이원텍,쎄노텍,코윈테크,이노메트리,SK이노베이션,LG화학,코스모신소재,피앤이솔루션,명성티엔에스,씨아이에스,엘앤에프,코스모화학,대보마그네틱,디에이테크놀로지,엠케이전자,웰크론한텍,상신이디피, 파워로직스\u200b회사별 간단하게 설명해논 링크 첨부합니다~모델S 사고 싶네요...\u200b2차전지 관련주 정리http://bit.ly/YIDINFORMAITON1500명이 사용하는여의도정보통 텔레그램 아시죠????실시간 빠...storyandstock.blog.me\u200b 더보기\nUPDOWN21\n댓글 [...] 더보기 \u200b안녕하세요,\u200b자동차 배터리나 기타 충전지는 2차 전지에 속하며 2차 전지 관련 기업 리스트를\u200b소개시켜 드리겠습니다.\u200b2차전지2차전지(Secondary battery)란 한 번 쓰고 버리는 배터리가 아닌 재충전이 가능한 배터리(rechargeable battery)를 의미함. 즉, 충전과 방전을 반복할 수 있는 전지임. 2차전지는 스마트폰, 태블릿PC, 노트북 등 모바일/휴대용 IT기기의 성장과 함께했으며, 최근에는 전기차/하이브리드카 등 중대형전지 시장으로 영역이 확대되고 있음.관련 

### **Image 생성 도구 (DALL-E)**

- `DallEAPIWrapper 클래스`: OpenAI의 DALL-E 이미지 생성기를 위한 래퍼(wrapper)

- 이 도구를 사용하면 DALL-E API를 쉽게 통합하여 텍스트 기반 이미지 생성 기능을 구현할 수 있음

- 다양한 설정 옵션을 통해 유연하고 강력한 이미지 생성 도구로 활용할 수 있음

**주요 속성**

- `model`: 사용할 DALL-E 모델 이름 (기본값: "dall-e-2", "dall-e-3")

- `n`: 생성할 이미지 수 (기본값: 1)

- `size`: 생성할 이미지 크기
  - "dall-e-2": "1024x1024", "512x512", "256x256"
  - "dall-e-3": "1024x1024", "1792x1024", "1024x1792"

- `style`: 생성될 이미지의 스타일 (기본값: "natural", "vivid")

- `quality`: 생성될 이미지의 품질 (기본값: "standard", "hd")

- `max_retries`: 생성 시 최대 재시도 횟수

**주요 기능**
- DALL-E API를 사용하여 텍스트 설명에 기반한 이미지 생성

In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.9, max_tokens=1000)

# DALL-E 이미지 생성을 위한 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    "Generate a detailed IMAGE GENERATION prompt for DALL-E based on the following description. "
    "Return only the prompt, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the prompt"
    "Output should be less than 1000 characters. Write in English only."
    "Image Description: \n{image_desc}",
)

# 프롬프트, LLM, 출력 파서를 연결하는 체인 생성
chain = prompt | llm | StrOutputParser()

# 체인 실행
image_prompt = chain.invoke(
    {"image_desc": "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"}
)

# 이미지 프롬프트 출력
print(image_prompt)

Create a neo-classical painting that humorously critiques modern society's obsession with smartphones. The scene should depict a diverse group of people from different backgrounds, all engrossed in their smartphones, ignoring their surroundings. The individuals should be dressed in elegant, classical attire reminiscent of the neo-classical era, blending historical fashion with contemporary technology. Include elements of classical architecture in the background, such as grand columns and intricate sculptures, to enhance the contrast between the old and the new. Use warm, rich colors and soft lighting to create a dramatic, contemplative atmosphere, while also incorporating subtle comedic elements, like exaggerated facial expressions of distraction and fascination with their devices.


In [ ]:
# # DALL-E API 래퍼 가져오기
# from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
# from IPython.display import Image
# import os

# # DALL-E API 래퍼 초기화
# # model: 사용할 DALL-E 모델 버전
# # size: 생성할 이미지 크기
# # quality: 이미지 품질
# # n: 생성할 이미지 수
# dalle = DallEAPIWrapper(
#     model="dall-e-3",
#     size="1024x1024",
#     quality="standard",
#     n=1,
# )

# # 질문
# query = "스마트폰을 바라보는 사람들을 풍자한 neo-classicism painting"

# # 이미지 생성 및 URL 받기
# # chain.invoke()를 사용하여 이미지 설명을 DALL-E 프롬프트로 변환
# # dalle.run()을 사용하여 실제 이미지 생성
# image_url = dalle.run(chain.invoke({"image_desc": query}))

# # 생성된 이미지를 표시합니다.
# Image(url=image_url, width=500)

### **사용자 정의 도구(Custom Tool)**

LangChain 에서 제공하는 빌트인 도구 외에도 사용자가 직접 도구를 정의하여 사용할 수 있음

이를 위해서는 `langchain.tools` 모듈에서 제공하는 `tool` 데코레이터를 사용하여 함수를 도구로 변환함

### **@tool 데코레이터**

이 데코레이터는 함수를 도구로 변환하는 기능을 제공함, 다양한 옵션을 통해 도구의 동작을 커스터마이즈할 수 있음

**사용 방법**
1. 함수 위에 `@tool` 데코레이터 적용
2. 필요에 따라 데코레이터 매개변수 설정

이 데코레이터를 사용하면 일반 Python 함수를 강력한 도구로 쉽게 변환할 수 있으며, 자동화된 문서화와 유연한 인터페이스 생성이 가능함

In [ ]:
# 두개 같음
from langchain.tools import tool
from langchain.agents import tool



# 데코레이터를 사용하여 함수를 도구로 변환
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

In [36]:
# 도구 실행
add_numbers.invoke({"a": 3, "b": 4})

7

In [37]:
# 도구 실행
multiply_numbers.invoke({"a": 3, "b": 4})

12

In [40]:
from langchain_teddynote.tools import GoogleNews

news_tool = GoogleNews()

news_tool.search_latest(k=5)

news_tool.search_by_keyword("AI 기술", k=3)

[{'url': 'https://news.google.com/rss/articles/CBMiYEFVX3lxTE1sUFFzR3UzSFhzR3FZVk1FenZvLWlNM0FvQWIxOUJ0b3pnOFZDZlozaUltT0pMakluTS1Sem9CRXN2MXhJYmlGUUdvY0xmaVJXZFhESzBmMmx6SFY5MjJTRtIBeEFVX3lxTFBKZUxTYWREUDdRQm9kX184UHpDbnhvajhsdDhNSWp4ZFdERm5pb1JIN0hvSkZFZUdjZnllRy0xdGRjZ2ZDbVpueWQ4c3J5RW9qWHBlSXI5TXRFQU4wSlp0UGJwMUtHU0pmRWU0cllsamxBcUpiZ0ttSw?oc=5',
  'content': '제조업 \'빨간불\'…"AI로 고용 유지하고 기술 유출 방지해야" - 뉴시스'},
 {'url': 'https://news.google.com/rss/articles/CBMiTkFVX3lxTE5fNHVxb3lPWVVmdjRsd2s3ZHZqS3Y2VktRT0JPTF9zLUdIajg5cm5yOHUzSTh0VVNoYjR3RzhJRk9mVjJmTm9QcHVBdWVxUQ?oc=5',
  'content': '김동욱 서울시의원, AI 기술의 윤리적 활용 조례 발의 - 전자신문'},
 {'url': 'https://news.google.com/rss/articles/CBMiU0FVX3lxTFBjRnRQdm5OdWo2b2xtcTZqNzRGem1SZGJybi1UR2poMTJsaVMxbFNsclZ4WjBIWmhIQWYybDhQVE1MQWFHUzhtWjVsSlZ6emVfR3F3?oc=5',
  'content': 'AI 기술이 진짜 흘러 들어가야 할 곳 - 네이트 뉴스'}]

In [44]:
from langchain_teddynote.tools import GoogleNews
from langchain.tools import tool
from typing import List, Dict


# 키워드로 뉴스 검색하는 도구 생성
@tool
def search_keyword(query: str) -> List[Dict[str, str]]:
    """Look up news by keyword"""
    # print(query)
    news_tool = GoogleNews()
    return news_tool.search_by_keyword(query, k=5)

search_keyword.invoke({"query": "LangChain AI"})

[{'url': 'https://news.google.com/rss/articles/CBMibkFVX3lxTE1qMC1EYjRZOEpMNVBEVVpSNm1BX3A4a3FrdmEyOTdDNWdjMGR1ZEgzWlpHZnRFZ1pSUXdKa3JnOXBQZU5qU3dKS3Q5VUNwLVZxRFZmbG1SZzJVSUoyM2lJbnZRbzhYYUJJcnFHWmp3?oc=5',
  'content': '랭체인 LangChain 이란 무엇인가? | 인사이트리포트 | 삼성SDS - Samsung SDS'},
 {'url': 'https://news.google.com/rss/articles/CBMiXEFVX3lxTE1FejJlRDF0QXZwME9YMHhGZkxEOXpKS045ZWhISFRYRHJWNDIyRlM4bk5MUXNTanA4T2JtWVF1aXhTcE5ReVFaR3M1ZDVJemtteE9UMVdRTkxCRk9u?oc=5',
  'content': 'LangChain이란 무엇인가요? - IBM'},
 {'url': 'https://news.google.com/rss/articles/CBMiSkFVX3lxTFB5Z3Y4SldkZm9maVdzQUhSTFBaTXZHUV8zbXRQem1mUUdmQkQ2VUhZZU1fTmVUc01WNldlMnl2X1RCVEdQUUVSYnB3?oc=5',
  'content': '20화 19. RAG(검색증강생성) 지원 - 브런치'},
 {'url': 'https://news.google.com/rss/articles/CBMilwFBVV95cUxNUklRWmZNZ3Y0ektzRm9scVR1UmI0Q0Z6NFh0MWxBYTlsOEFDdTBzYUY4UERva19HSnBLdElYVHBRbjV6NU52MDJtQ0xZOVlXb2VCak5VSVRrZ1FRMFZTaUNqUTBPMnN6cXVlWi1nUWNWNXZrOXRzWjVrZThQYXZBQjV6MG1EMmdxR2V3NHhUWW84ZDU3YWFB?oc=5',
  'content': 'NVIDIA와 LangCh